In [17]:
import db.db_queries_webtraffic_sales as db
import db.db_access as access
import web_traffic_sales.process as pr
import mysql.connector
import pandas as pd

In [3]:
db_obj = db.TrafficDatabaseConnector()
all_reported_sales = db.cache_db_request_pandas(db_obj.get_reported_sales_pandas_2,
                                                '../../db/queries/get_reported_sales_pandas.db')
all_reported_sales

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
0,43,1,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,44,1,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,45,1,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,46,1,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,47,1,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...,...
20680,17793,534,18q4,2018-12-31,2018-10-01,2019-02-14,9.107400e+07,9.107400e+07,16:03:00
20681,17794,534,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
20682,17795,534,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
20683,17796,534,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00


In [26]:
report_sales_zero = all_reported_sales[all_reported_sales['actual_sales']==0]
report_sales_zero

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
363,4648,8,13q1,2011-09-30,2011-07-02,2013-10-15,0.0,4.940970e+07,00:00:00
364,4650,8,13q1,2011-12-31,2011-10-01,2013-10-15,0.0,6.531200e+07,00:00:00
365,4652,8,13q1,2012-03-31,2012-01-01,2013-10-15,0.0,7.409520e+07,00:00:00
366,4654,8,13q1,2012-06-30,2012-04-01,2013-10-15,0.0,7.269350e+07,00:00:00
367,4656,8,13q1,2012-09-30,2012-07-01,2013-10-15,0.0,9.024980e+07,00:00:00
...,...,...,...,...,...,...,...,...,...
20457,20479,527,16q4,2016-02-29,2016-01-01,2017-02-07,0.0,1.303000e+09,16:30:00
20471,20495,528,11q2,2011-06-30,2011-04-01,2011-08-24,0.0,1.584500e+07,16:00:00
20506,20530,529,14q1,2014-03-31,2014-01-01,2014-05-09,0.0,5.010000e+08,00:00:00
20556,20580,531,06q1,2006-06-30,2006-01-01,2006-08-17,0.0,2.059600e+10,00:00:00


In [33]:
query = 'UPDATE reported_sales SET actual_sales = %s WHERE (id_reported_sales = %s);'

In [44]:
def update_reported_sales(df):
    cnx = mysql.connector.connect(user='patrick', password='M@ch1neTallinn', host='35.228.108.97', database='altdata')
    cur = cnx.cursor()
    
    for i in df.index:
        sales = df.at[i, 'actual_sales']
        id_sales = df.at[i, 'id_reported_sales']
        data = (float(sales), int(id_sales))
        cur.execute(query, data)
        cnx.commit()
        
    cnx.close()
    cur.close()

In [45]:
update_reported_sales(df=report_sales_zero)

In [46]:
all_reported_sales = db.cache_db_request_pandas(db_obj.get_reported_sales_pandas_2,
                                                '../../db/queries/get_reported_sales_pandas.db')

report_sales_zero = all_reported_sales[all_reported_sales['actual_sales']==0]
report_sales_zero

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
363,4648,8,13q1,2011-09-30,2011-07-02,2013-10-15,0.0,4.940970e+07,00:00:00
364,4650,8,13q1,2011-12-31,2011-10-01,2013-10-15,0.0,6.531200e+07,00:00:00
365,4652,8,13q1,2012-03-31,2012-01-01,2013-10-15,0.0,7.409520e+07,00:00:00
366,4654,8,13q1,2012-06-30,2012-04-01,2013-10-15,0.0,7.269350e+07,00:00:00
367,4656,8,13q1,2012-09-30,2012-07-01,2013-10-15,0.0,9.024980e+07,00:00:00
...,...,...,...,...,...,...,...,...,...
20457,20479,527,16q4,2016-02-29,2016-01-01,2017-02-07,0.0,1.303000e+09,16:30:00
20471,20495,528,11q2,2011-06-30,2011-04-01,2011-08-24,0.0,1.584500e+07,16:00:00
20506,20530,529,14q1,2014-03-31,2014-01-01,2014-05-09,0.0,5.010000e+08,00:00:00
20556,20580,531,06q1,2006-06-30,2006-01-01,2006-08-17,0.0,2.059600e+10,00:00:00


In [40]:
def update_reported_sales(df):
    cnx = mysql.connector.connect(user='patrick', password='M@ch1neTallinn', host='35.228.108.97', database='altdata')
    cur = cnx.cursor()
    df = df[['actual_sales', 'id_reported_sales']]
    cur.execute(query, df.values.tolist())
    cnx.commit()

(100, 11)

In [47]:
all_reported_sales = db.cache_db_request_pandas(db_obj.get_reported_sales_pandas_2,
                                                '../../db/queries/get_reported_sales_pandas.db')

In [48]:
for i in all_reported_sales.index:
    if all_reported_sales.at[i, 'actual_sales'] == 0:
        all_reported_sales.at[i, 'actual_sales'] = all_reported_sales.at[i, 'sales_reported']
        
all_reported_sales

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
0,43,1,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,44,1,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,45,1,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,46,1,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,47,1,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...,...
20680,17793,534,18q4,2018-12-31,2018-10-01,2019-02-14,9.107400e+07,9.107400e+07,16:03:00
20681,17794,534,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
20682,17795,534,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
20683,17796,534,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00


In [49]:
all_reported_sales[all_reported_sales['actual_sales']==0]

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
860,861,20,17q2,2016-06-30,2016-04-01,2017-08-10,0.0,0.0,07:15:00
861,862,20,17q2,2016-09-30,2016-07-01,2017-08-10,0.0,0.0,07:15:00
1447,1448,35,04q4,2005-01-01,2004-10-03,2005-03-10,0.0,0.0,00:00:00
1721,1722,39,14q4,2014-09-24,2014-07-03,2014-11-24,0.0,0.0,06:05:00
2260,2261,51,14q1,2013-03-30,2012-12-30,2014-09-29,0.0,0.0,00:00:00
3583,3570,82,10q1,2008-06-30,2008-04-01,2010-05-06,0.0,0.0,00:00:00
3584,3571,82,10q1,2008-09-30,2008-07-01,2010-05-06,0.0,0.0,00:00:00
3585,3572,82,10q1,2008-12-31,2008-10-01,2010-05-06,0.0,0.0,00:00:00
5383,5511,127,08q2,2007-08-03,2007-05-05,2008-09-03,0.0,0.0,00:00:00
5386,5510,127,09q2,2007-08-03,2007-05-05,2008-09-03,0.0,0.0,00:00:00


In [57]:
all_reported_sales = all_reported_sales.drop('id_reported_sales', axis=1)


KeyError: "['id_reported_sales'] not found in axis"

In [59]:
all_reported_sales.to_csv('reported_sales.csv',index=False)

,id_reported_sales,company_id,group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time
0,43,1,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,44,1,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,45,1,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,46,1,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,47,1,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...,...
20680,17793,534,18q4,2018-12-31,2018-10-01,2019-02-14,9.107400e+07,9.107400e+07,16:03:00
20681,17794,534,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
20682,17795,534,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
20683,17796,534,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00
